# ZCTA Proxy for Broward County, FL

This is an adaptation of the Dept. of Treasury [Guidelines for fact-specific proxies](https://home.treasury.gov/policy-issues/coronavirus/assistance-for-state-local-and-tribal-governments/emergency-rental-assistance-program/service-design/fact-specific-proxies) using ZIP Code Tabulation Areas for Lackawanna County, PA. Because the main [Data Pipeline for Emergency Rental Assistance](https://github.com/usds/erap/tree/main/arp_pipeline) is tract/address-based, this is a one-off. However, it follows the same basic conceptual outlines as the main pipeline, and so should be possible to integrate as an additonal output for states and counties if there's greater demand.

The steps are as follows:

- Lookup FL's FIPS code and Broward County's code as well
- Grab the census variables we use in the analysis (median income, median renter income) and do a little cleaning on the census data
- Use the [Census ZCTA-County mapping](https://www2.census.gov/geo/docs/maps-data/data/rel/zcta_county_rel_10.txt) to figure out which ZCTAs are in Lackawanna County, PA, and filter down the census data to just those ZCTAs
- Add in the HUD income limit for the county
- Backfill any missing renter incomes with median incomes for the ZCTA
- Do a comparison of the income-data we have with the income limit to determine our facts for each ZCTA
- Export

In [1]:
import censusdata
import numpy as np
import pandas as pd
from scipy import stats
import enum

In [2]:
%load_ext sql

In [3]:
state = 'Florida'
county = 'Broward'

In [4]:
state_geos = censusdata.geographies(
    censusdata.censusgeo([("state", "*")]), "acs5", 2019
)


In [5]:
statefp = [val for name, val in state_geos.items() if name==state][0].params()[0][-1]

In [6]:
statefp

'12'

In [7]:
%%sql postgresql://postgres:postgres@db/geocoder
select * from county where statefp = :statefp and name = :county

1 rows affected.


gid,statefp,countyfp,countyns,cntyidfp,name,namelsad,lsad,classfp,mtfcc,csafp,cbsafp,metdivfp,funcstat,aland,awater,intptlat,intptlon,the_geom
284,12,011,00295753,12011,Broward,Broward County,06,H1,G4020,370,33100,22744,A,3115289087,310801098.0,+26.1935353,-080.4766834,...


In [8]:
county_result = _

In [9]:
countyfp = county_result.DataFrame().iloc[0].countyfp

In [10]:
class CensusVars(str, enum.Enum):
    median_renter_income = 'B25119_003E'
    median_income = 'B19013_001E'
    households_count = 'B07013_001E'
    families_count = 'B17026_001E'
    homeowners_count = 'B07013_002E'
    renters_count = 'B07013_003E'

In [11]:
state_data = censusdata.download(
        "acs5",
        2019,
        censusdata.censusgeo([("state", statefp), ("zip code tabulation area", "*")]),
        [val.value for val in CensusVars],
)


In [12]:
NULL_SIGNALS = [-666666666, -333333333, -222222222]
state_data.replace(to_replace=NULL_SIGNALS, value=np.nan, inplace=True)
state_data.columns = [val.name for val in CensusVars]

In [13]:
zcta_county_map = pd.read_csv('zcta_county_rel_10.txt', dtype='str')

In [14]:
county_zctas = set(zcta_county_map[(zcta_county_map.COUNTY==countyfp) & (zcta_county_map.STATE==statefp)].ZCTA5)

In [15]:
state_data['zcta'] = state_data.index.map(lambda idx: idx.params()[-1][-1])

In [16]:
zip_data = state_data[state_data.zcta.isin(county_zctas)].copy()

In [17]:
zip_data.set_index('zcta', inplace=True)

In [18]:
hud = %sql select * from hud.income_limits where state_alpha = 'FL' and county_name ilike '{county} County'

 * postgresql://postgres:***@db/geocoder
1 rows affected.


In [19]:
hud = hud.DataFrame()

In [20]:
hud.iloc[0].l80_1

49300

In [21]:
zip_data['hud_income_limit'] = hud.iloc[0].l80_1

In [22]:
zip_data['comparison_income'] = zip_data.median_renter_income
zip_data.loc[zip_data.median_renter_income.isna(), 'comparison_income'] = zip_data.median_income

In [23]:
zip_data['does_income_qualify'] = 'False'
zip_data.loc[(~zip_data.comparison_income.isna()) & (zip_data.comparison_income < zip_data.hud_income_limit), 'does_income_qualify'] = 'True'

In [24]:
zip_data

,median_renter_income,median_income,households_count,families_count,homeowners_count,renters_count,hud_income_limit,comparison_income,does_income_qualify
zcta,,,,,,,,,
33308,48198.0,64792.0,27027.0,6438.0,19155.0,7872.0,49300,48198.0,True
33314,44647.0,51635.0,25795.0,5656.0,10007.0,15788.0,49300,44647.0,True
33317,39529.0,67878.0,37733.0,9246.0,28466.0,9267.0,49300,39529.0,True
33322,49574.0,48179.0,38665.0,9456.0,31349.0,7316.0,49300,49574.0,False
33325,63505.0,82805.0,29206.0,7143.0,22808.0,6398.0,49300,63505.0,False
33073,62989.0,77158.0,34891.0,8499.0,21501.0,13390.0,49300,62989.0,False
33306,48802.0,71357.0,3113.0,734.0,2152.0,961.0,49300,48802.0,True
33311,29656.0,36649.0,73924.0,14693.0,28749.0,45175.0,49300,29656.0,True
33326,64680.0,93197.0,32745.0,8531.0,21680.0,11065.0,49300,64680.0,False


In [26]:
zip_data[['does_income_qualify', 'hud_income_limit', 'median_renter_income', 'median_income']].to_excel(f'{county}-county-{state}.xlsx')